# Efficient FastAPI

## Introduction
FastAPI embraces many of the latest Python features and external libraries. Thus, writing APIs is relatively easy and less error-prone. These features include
1. Type hints
2. Async/await
3. ASGI servers
4. Pydantic classes

We recommended that you refresh your knowledge of Python type hinting and Pydantic classes before proceeding with this lesson.

FastAPI employs type hinting to transform query parameters into the specified data type provided as a hint.

## Async/Await: Asynchronous IO 

Async IO is a concurrent programming design that allows us to run multiple tasks that overlap. It should not be confused with multi-threading, since async IO is a single-threaded, single-process programming design. In this case, the term, asynchronous, refers to the ability to pause the execution of a task, awaiting a trigger, while other tasks run.

To understand better, see the following `synchronous` example:

In [10]:
import time

def count(x):
    print(x + 1)
    time.sleep(1)

def main():
    for i in range(3):
        count(i)

s = time.perf_counter()
main()
elapsed = time.perf_counter() - s
print(f"Executed in {elapsed:0.2f} seconds.")

The code above simply prints from 1 to 3, with a 1-s pause between numbers. Now, observe the following `asynchronous` example:

In [17]:
%%python
import asyncio
import time

async def count(x):
    print(x)
    await asyncio.sleep(1)

async def main():
    await asyncio.gather(count(1), count(2), count(3))


s = time.perf_counter()
asyncio.run(main())
elapsed = time.perf_counter() - s
print(f"Executed in {elapsed:0.2f} seconds.")

1
2
3
Executed in 1.00 seconds.


Observe the difference between the times. In the asynchronous example, there is no pause between 1, 2 and 3. The async count function is asynchronous, indicating that it can pause execution right before the `await` statement. Moreover, note that the asynchronous method is considerably faster than its synchronous counterpart. This is because there is no `dead time` in between all the concurrent calls.

Conversely, asyncio.gather creates three concurrent tasks (count(1), count(2) and count(3)). Although these tasks do not run simultaneously, they can overlap.

Below, we present a rough demonstration of the above example:
```
        import asyncio
        import time

        async def count(x):
            print(x)
(3, 5, 7)   await asyncio.sleep(1)

        async def main():
(2, 4, 6)   await asyncio.gather(count(1), count(2), count(3))


        s = time.perf_counter()
(1)     asyncio.run(main())                                     
        elapsed = time.perf_counter() - s
        print(f"Executed in {elapsed:0.2f} seconds.")
```

1. Run `main()`
2. Run `count(1)`
3. Print 1, and reach `await asyncio.sleep(1)`. Tells async to execute the next concurrent task, while this one pauses.
4. Run `count(2)`
5. Print 2, and reach `await asyncio.sleep(1)`. Tells async to execute the next concurrent task, while this one pauses.
6. Run `count(3)`
7. Print 3, and reach `await asyncio.sleep(1)`. Since there are no more concurrent jobs, the execution continues.



Similarly, most times, we wait when we scrape data from a website. This waiting period is, most times, for a single response, after which the next sample is extracted. You can exploit async and await using the `httpx` library. Visit the `examples` folder, where there are two examples: one asynchronous solution and its synchronous counterpart. You can experiment on them to determine their differences. 

> Asynchronous IO provides a considerably efficient solution for running processes by running concurrent subprocesses.

### ASGI Servers

Traditional web frameworks run sequentially. A request is sent, the server processes the request, and the response is returned. If we establish a set of rules for the communication between the user and the server, we will, summarily, obtain a __Web Server Gateway Interface__ (WSGI).

Currently, [Gunicorn](https://gunicorn.org/) is the most used HTTP WSGI for Python, and it works very well with different Python frameworks, including Flask and Django, when deploying apps for production.

However, it is possible to work asynchronously, send multiple requests simultaneously, and subsequently process each of them. A server that works asynchronously is referred to as an __Asynchronous Server Gateway Interface__ (ASGI), which leverages `async` and `await`.

Currently, [Uvicorn](https://www.uvicorn.org/) is one of the main servers that support ASGI. To explore the different frameworks that can work in an ASGI server, check out the information [here](https://github.com/florimondmanca/awesome-asgi).

<p align=center><img src=images/ASGI.png width=500></p>



## Routing with FastAPI

Conventionally, when working on a project, the code is partitioned into smaller scripts to ensure organisation. In FastAPI, this can be achieved using Routers. As an example, consider a case where you add two GET responses to your API: one for a regular DOB request and another for supplying an icon to the tab on which you are working:

In [ ]:
%%python
import fastapi
import uvicorn

api = fastapi.FastAPI()

@api.get('/')
def index():
    return 'Welcome to the celebrity DOB API!'

@api.get('/api/dob')
def dob():
    return 'Some random DOB'
    
if __name__ == '__main__':
    uvicorn.run(api, port=8008, host='127.0.0.1')

To view the output, visit `http://127.0.0.1:8008/api/dob`.
<p align=center><img src=images/Routing_1.png width=300></p>

One approach for separating these functions would be having one script for the home page and another for additional pages (`/api/dob`):

```
root/
│
├── home.py
├── api
│   └── dob_api.py
└── main.py
```

In [ ]:
# views/home.py

import fastapi

router = fastapi.APIRouter()


@router.get('/')
def index():
    return 'Welcome to the celebrity DOB API!'


In [ ]:
# api/dob_api.py

import fastapi

router = fastapi.APIRouter()

@router.get('/api/dob')
def dob():
    return 'Some random DOB'

In [ ]:
# main.py
import fastapi
import uvicorn
import home

from api import dob_api

api = fastapi.FastAPI()


def configure_routing():
    api.include_router(home.router)
    api.include_router(dob_api.router)


if __name__ == '__main__':
    configure_routing()
    uvicorn.run(api, port=8000, host='127.0.0.1')


Observe that in `home.py` and `api/dob_api.py`, we included `router = fastapi.APIRouter()`. As a result, all the files have a common node to search for all requests.

## Exercise

1. Create a directory named views.
2. Add a file named home.py in said directory.
3. Add the code above.
4. Create a directory named api.
5. Add a file named dob_api.py in said directory.
6. Add the code above to dob_api.py.
7. Create a file named main.py, and add the code above.
8. Run the code, and ensure that everything works.
9. Tweak dob_api so that `/api/dob` can accept two query parameters: first name and last name.

## Introducing the Pydantic Model

APIs accept request parameters, and the Pydantic model is responsible for casting these parameters to the correct type. In the above exercise, your built API should accept the first name and last name as parameters. 

Pydantic allows you to create a class where each attribute can be cast to a specified data type.

In [4]:
from pydantic import BaseModel
from typing import Optional

class Celebrity(BaseModel):
    '''
    This class gives some information about a celebrity. It is intended to be 
    used with the FastAPI example.

    Attributes
    ----------
    first_name: str
        The first name of the celebrity
    last_name: str
        The last name of the celebrity
    city: bool
        If True, the API will also return the city where the celebrity was born.
    '''
    first_name: str
    last_name: str
    city: bool = False

uma_thurman = Celebrity(first_name='Uma', last_name='Thurman')    

Now, we create another directory to store our models:

```
root/
│
├── home.py
├── api
│   └── dob_api.py
├── models
│   └── celebrities.py
└── main.py
```

Thereafter, this Pydantic model can be employed in our FastAPI, as shown below:

In [ ]:
# models/celebrities.py

from pydantic import BaseModel
from typing import Optional

class Celebrity(BaseModel):
    '''
    This class gives some information about a celebrity. It is intended to be 
    used with the FastAPI example.

    Attributes
    ----------
    first_name: str
        The first name of the celebrity
    last_name: str
        The last name of the celebrity
    city: bool
        If True, the API will also return the city where the celebrity was born.
    '''
    first_name: str
    last_name: str
    city: bool = False

In [ ]:
# api/dob_api.py

import fastapi
from fastapi import Depends

from models.celebrities import Celebrity

router = fastapi.APIRouter()


@router.get('/api/dob/{first_name}')
def dob(celebrity: Celebrity = Depends()):
    full_name = f'{celebrity.first_name} {celebrity.last_name}'
    if celebrity.city:
        report = f'{full_name} was born a random day in a random city'
    else:
        report = f'{full_name} was born a random day'
    return report



Observe the `Depends` class in `celebrity`, used as an argument in `dob`. This tells the dob function that the variables it contains depend on an external class; therefore, it becomes aware that the attributes of the class are to be used as the query parameters.

With this, the following URL [http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman](http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman) will return the following:

<p align=center><img src=images/Routing_2.png width=400></p>

Currently, the DOB API is not meeting expectations. Thus, we create the real function that will scrape the data on the celebrity and return the date through our API. The function containing this service will be in the `services` directory:

```
root/
│
├── home.py
├── api
│   └── dob_api.py
├── models
│   └── celebrities.py
├── services
│   └── dob_service.py
└── main.py
```

In [19]:
# services/dob_service.py
import requests
from bs4 import BeautifulSoup
import json
import re

def get_dob(first_name: str, last_name: str, city: bool=False):
    infobox_data = get_infobox(first_name, last_name)
    if not infobox_data:
        return None
    birthday = infobox_data.find('span', {'class': 'bday'})

    report = {'first name': first_name,
              'last_name': last_name,
              'Date of Birth': birthday.text}
    if city:
        birthplace = infobox_data.find('div', {'class': 'birthplace'})
        report['City'] = birthplace.text

    return report

def get_infobox(first_name: str, last_name: str):
    r = requests.get(f'https://en.wikipedia.org/wiki/{first_name}_{last_name}')
    soup = BeautifulSoup(r.text, 'html.parser')
    if soup.find_all('b', text=re.compile('Wikipedia does not have an article with this exact name')):
        return None
    celeb_infobox = soup.find('table', {'class': 'infobox biography vcard'})
    return celeb_infobox.find('td', {'class': 'infobox-data'})

In [28]:
# api/dob_api.py

import fastapi
from fastapi import Depends
import json
from models.celebrities import Celebrity
# from services.dob_service import get_dob
from services.dob_service_async import get_dob
router = fastapi.APIRouter()

@router.get('/api/dob/{first_name}')
def dob(celebrity: Celebrity = Depends()):
    report = get_dob(celebrity.first_name, celebrity.last_name, celebrity.city)
    return fastapi.Response(content=json.dumps(report),
                            media_type='application/json')

If you visit the following URL [http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman&city=True](http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman&city=True), you will obtain a real API response.
<p align=center><img src=images/Routing_3.png width=500></p>

Further, the requests library can be employed to retrieve the uploaded JSON object and guarantee that everything works properly.

In [30]:
requests.get('http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman&city=True').json()

{'First name': 'Uma',
 'Last_name': 'Thurman',
 'Date of Birth': '1970-04-29',
 'City': 'Boston, Massachusetts, U.S.'}

## Async/Await Implementation

As shown above, FastAPI leverages modern Python tools, particularly asynchronous functions. To implement these tools, we simply need to change the `dob_service.py` file. Observe the implemented changes in the code below, considering that requests do not work asynchronously.

In [ ]:
# services/dob_service.py

from bs4 import BeautifulSoup
import json
import re
import httpx
async def get_dob(first_name: str, last_name: str, city: bool=False):
    infobox_data = await get_infobox_async(first_name, last_name)
    if not infobox_data:
        return None
    birthday = infobox_data.find('span', {'class': 'bday'})

    report = {'first name': first_name,
              'last_name': last_name,
              'Date of Birth': birthday.text}
    if city:
        birthplace = infobox_data.find('div', {'class': 'birthplace'})
        report['City'] = birthplace.text

    return report

async def get_infobox_async(first_name: str, last_name: str):
    async with httpx.AsyncClient() as client:
        r = await client.get(f'https://en.wikipedia.org/wiki/{first_name}_{last_name}')
        html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    if soup.find_all('b', text=re.compile('Wikipedia does not have an article with this exact name')):
        return None
    celeb_infobox = soup.find('table', {'class': 'infobox biography vcard'})
    return celeb_infobox.find('td', {'class': 'infobox-data'})

In [ ]:
# api/dob_api.py

import fastapi
from fastapi import Depends
import json
from models.celebrities import Celebrity
from services.dob_service import get_dob


router = fastapi.APIRouter()

@router.get('/api/dob/{first_name}')
async def dob(celebrity: Celebrity = Depends()):
    report = await get_dob(celebrity.first_name, celebrity.last_name, celebrity.city)
    return fastapi.Response(content=json.dumps(report),
                                media_type='application/json')

As observed above, whenever an asynchronous function is called, we need to `await` it. Similarly, if we must use `await` because we are calling an asynchronous function, we must include it in an `async` function.

The output will be the same as that obtained previously; however, when this API is deployed, it will provide many more responses at a considerably high speed.

## Conclusion
At this point, you should have a good understanding of
- how to leverage modern Python tools in FastAPI:
    - pydantic
    - async and await
- the difference between ASGI and WSGI servers.
- how to route API requests to a single node called in main.